In [2]:
from datapreparation import *
from simsiam import *
from utils import *
from evaluation import *
import torch.optim


# Train SimSiam

In [7]:
trainloader, testloader = prepare_data()

Files already downloaded and verified
Files already downloaded and verified


In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = SimSiam()
model = model.to(device)

/home/jonas/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9, weight_decay=0.0005)

In [ ]:
for epoch in range(5):  # loop over the dataset multiple times
    epoch_loss = 0.0
    running_loss = 0.0
    for i, data in enumerate(trainloader):            
        # get the inputs; data is a list of [inputs, labels]
        # inputs, labels = data
        images, labels = data[0], data[1].to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # get the two views (with random augmentations):
        x1 = images[0].to(device)
        x2 = images[1].to(device)
        
        # forward + backward + optimize
        z1, p1 = model(x1)
        z2, p2 = model(x2)
        #loss = criterion(outputs, labels)
        loss = D(p1, z2)/2 + D(p2, z1)/2
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        epoch_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0
    print("epoch loss = ", epoch_loss/len(trainloader))
print('Finished Training')

In [ ]:
PATH = "models/simsiam.pth"

# save model
# torch.save(model.state_dict(), PATH)

# load trained model
model = SimSiam()
model.load_state_dict(torch.load(PATH, map_location=device))

In [ ]:
visualize_knn(0)

# SSL vs Supervised Comparison on 1%/10% of data

## Linear Evaluation

In [10]:
trainloader, testloader = get_downstream_data(percentage_of_data=0.1, batch_size=4)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### Supervised model

In [12]:
model = SupervisedModel(pretrained=True, linearevaluation=True)
model = model.to(device)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [14]:
train_downstream(5, model, trainloader, criterion, optimizer, device)

KeyboardInterrupt: 

In [ ]:
evaluate_downstream(model, testloader, device)

### SimSiam

In [16]:
model = SimSiamDownstream(trained_model_path='./models/simsiam.pth', device=device, linearevaluation=True)
model = model.to(device)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [18]:
train_simsiam_downstream(5, model, trainloader, criterion, optimizer, device)

KeyboardInterrupt: 

In [ ]:
evaluate_simsiam_downstream(model, testloader, device)

# Fine Tuning
### SimSiam vs Supervised Learning on 1% / 10% of data

#### Supervised model

In [ ]:
model = SupervisedModel(pretrained=True, linearevaluation=False)
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
train_downstream(5, model, trainloader, criterion, optimizer, device)

In [ ]:
evaluate_downstream(model, testloader, device)

### SimSiam

In [ ]:
model = SimSiamDownstream(trained_model_path='../models/simsiam.pth', device=device, linearevaluation=False)
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
train_simsiam_downstream(5, model, trainloader, criterion, optimizer, device)

In [ ]:
evaluate_simsiam_downstream(model, testloader, device)